# Assignment

## Part 1- Web Scraping

In [2]:
#import required libs
import requests
import pandas as pd
from bs4 import BeautifulSoup

## Extracting table from web page using BeautifulSoup

In [4]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')
table = soup.find('table',{'class':'wikitable sortable'})


### Start reading table raws into arrays

In [5]:
links=table.findAll('th')

Postalcode=[]
Borough=[]
Neighbourhood=[]

for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        Postalcode.append(cells[0].find(text=True))
        Borough.append(cells[1].find(text=True))
        Neighbourhood.append(cells[2].find(text=True))
        
print(Postalcode, Borough, Neighbourhood)

['M1A', 'M2A', 'M3A', 'M4A', 'M5A', 'M5A', 'M6A', 'M6A', 'M7A', 'M8A', 'M9A', 'M1B', 'M1B', 'M2B', 'M3B', 'M4B', 'M4B', 'M5B', 'M5B', 'M6B', 'M7B', 'M8B', 'M9B', 'M9B', 'M9B', 'M9B', 'M9B', 'M1C', 'M1C', 'M1C', 'M2C', 'M3C', 'M3C', 'M4C', 'M5C', 'M6C', 'M7C', 'M8C', 'M9C', 'M9C', 'M9C', 'M9C', 'M1E', 'M1E', 'M1E', 'M2E', 'M3E', 'M4E', 'M5E', 'M6E', 'M7E', 'M8E', 'M9E', 'M1G', 'M2G', 'M3G', 'M4G', 'M5G', 'M6G', 'M7G', 'M8G', 'M9G', 'M1H', 'M2H', 'M3H', 'M3H', 'M3H', 'M4H', 'M5H', 'M5H', 'M5H', 'M6H', 'M6H', 'M7H', 'M8H', 'M9H', 'M1J', 'M2J', 'M2J', 'M2J', 'M3J', 'M3J', 'M4J', 'M5J', 'M5J', 'M5J', 'M6J', 'M6J', 'M7J', 'M8J', 'M9J', 'M1K', 'M1K', 'M1K', 'M2K', 'M3K', 'M3K', 'M4K', 'M4K', 'M5K', 'M5K', 'M6K', 'M6K', 'M6K', 'M7K', 'M8K', 'M9K', 'M1L', 'M1L', 'M1L', 'M2L', 'M2L', 'M3L', 'M4L', 'M4L', 'M5L', 'M5L', 'M6L', 'M6L', 'M6L', 'M7L', 'M8L', 'M9L', 'M1M', 'M1M', 'M1M', 'M2M', 'M2M', 'M3M', 'M4M', 'M5M', 'M5M', 'M6M', 'M6M', 'M6M', 'M6M', 'M7M', 'M8M', 'M9M', 'M9M', 'M1N', 'M1N', 'M2N'

In [6]:
df=pd.DataFrame()
df['Postalcode']= Postalcode
df['Borough']= Borough
df['Neighbourhood']= Neighbourhood
df.head()

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 3 columns):
Postalcode       288 non-null object
Borough          288 non-null object
Neighbourhood    288 non-null object
dtypes: object(3)
memory usage: 6.8+ KB
None


In [7]:
df.drop(df.loc[df['Borough'] == 'Not assigned'].index, inplace=True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 211 entries, 2 to 286
Data columns (total 3 columns):
Postalcode       211 non-null object
Borough          211 non-null object
Neighbourhood    211 non-null object
dtypes: object(3)
memory usage: 6.6+ KB
None


### Merge raws with same postalcode

In [8]:
df=df.sort_values(by=['Postalcode','Neighbourhood'], ascending=False)
df2 = (df.groupby(['Postalcode','Borough'])['Neighbourhood'].apply(lambda x: list(set(x))).reset_index()).sort_values(by=['Postalcode']) 


#convert list to string values
for i in df2['Neighbourhood'].index:
    df2['Neighbourhood'][i] =','.join(df2['Neighbourhood'][i])
    df2['Neighbourhood'][i] = df2['Neighbourhood'][i].replace('\n', '');
    
df2.head()

,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Port Union,Rouge Hill"
2,M1E,Scarborough,"West Hill,Guildwood,Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Replace  neighborhood  'Not assigned' values by  borough values

In [9]:
df2.loc[df2['Neighbourhood'] == 'Not assigned','Neighbourhood']=df2['Borough']


### Number of rows:

In [10]:
df2.shape

(103, 3)

## Part 2

### Importing cordinates from csv

In [35]:
url='http://cocl.us/Geospatial_data'
coordinates=pd.read_csv(url)
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


removing space in column name'Postal Code'

In [36]:
coordinates.rename(columns={'Postal Code':'Postalcode'},inplace=True)
coordinates.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [37]:
coordinates.shape

(103, 3)

### Merging postal code, neighbourhoods dataframe with cordinates dataframe

In [39]:
df_merged=pd.merge(df2, coordinates, on='Postalcode', how='inner')
df_merged.head(11)

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Port Union,Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"West Hill,Guildwood,Morningside",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Ionview,Kennedy Park,East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Scarborough Village West,Cliffcrest,Cliffside",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


## Part 3

## Filterig dataframe by boroughs that contain the word Toronto

In [51]:
neighborhoods=df_merged[df_merged['Borough'].str.contains('Toronto')]
neighborhoods.reset_index(drop=True, inplace=True)
neighborhoods.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


## Clustering the neighborhoods of Toronto

In [31]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'toronto, canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of toronto are 43.653963, -79.387207.


In [58]:
# create map of New York using latitude and longitude values
import folium # map rendering library
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhoods, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto